# ILAMB
## Manual instructions

**NOTE** This example assumes you already have regular, lat-lon, data (e.g., from an f09 case).  CUPiD tools to handle regridding history data from an unstructured grid are a work in progress.

[Sample output from ILAMB is in this link](https://webext.cgd.ucar.edu/I20TR/ctsm5.4.004_bgc_f09_131_HIST/lnd/computed_notebooks/_build/html/infrastructure/index.html).

ILAMB can be run in CUPiD users via the following process:
### 1) Set up the CUPiD configuration file

   * `cd CUPiD/examples/land_only`
   * `conda activate cupid-infrastructure`

Set a softlink to the config file you want to use
   #### 1a) _This example will create a subset of ILAMB output for quicker analyses_
  
   * `ln -s config_subset.yml config.yml`

   #### 1b)_OR for the full ILAMB output on a BGC case:_

   * `ls -s config_full.yml config.yml`

Modify the `config.yml` file (if needed) to point to the cases you want to run

### 2) Generate ILAMB config files based on a CUPiD configuration file.

This step uses _CUPiD/helper_scripts/generate_ilamb_config_files.py_ 
 
   #### 2a)_This example will create a subset of ILAMB output for quicker analyses_ (takes about 30 minutes on the 4x5 resolution example provided) 

   * `../../helper_scripts/generate_ilamb_config_files.py --cupid-config-loc . --run-type BGC_subset --cupid-root ../../`

   #### 2b)_OR for the full ILAMB output on a BGC case:_ (takes about 90 minutes on the 4x5 resolution example provided)

   * `../../helper_scripts/generate_ilamb_config_files.py --cupid-config-loc . --run-type BGC --cupid-root ../../`

   #### 2c)_OR for the full ILAMB output on an SP case:_ (not tested at this time)

   * `../../helper_scripts/generate_ilamb_config_files.py --cupid-config-loc . --run-type SP --cupid-root ../../`

**HINT:** additional model confrontations can be manually added to the `model_setup.txt` file.

**NOTE:** ILAMB can handle raw (h0) history files or single variable timeseries!

### 3) Launch an interactive job on casper
   * `export PBS_ACCOUNT=P#########` (e.g. P93300041 for the LMWG)
   * `qinteractive -I-X  -l select=1:ncpus=16:mpiprocs=16:mem=100G -q casper@casper-pbs -l walltime=08:00:00 -A $PBS_ACCOUNT`

### 4) Run ILAMB with the newly created configuration file following the printed instructions

**NOTE:** If an ILAMB_output dir already exists in CUPiD/examples/land_only, remove it

   * `export ILAMB_ROOT=../../ilamb_aux`
   * `conda activate cupid-analysis`

Below we're also assuming your running a comparison with newer `h0a` history files

   #### 4a) _This example will create a subset of ILAMB output for quicker analyses._  

   * `mpiexec ilamb-run --config ilamb_nohoff_final_CLM_BGC_subset.cfg --build_dir ILAMB_output/ --df_errs ${ILAMB_ROOT}/quantiles_Whittaker_cmip5v6.parquet --define_regions ${ILAMB_ROOT}/DATA/regions/LandRegions.nc ${ILAMB_ROOT}/DATA/regions/Whittaker.nc --regions global --model_setup model_setup.txt --filter .clm2.h0a`

   #### 4b)_OR for the full ILAMB output on a BGC case:_

   * `mpiexec ilamb-run --config ilamb_nohoff_final_CLM_BGC.cfg --build_dir ILAMB_output/ --df_errs ${ILAMB_ROOT}/quantiles_Whittaker_cmip5v6.parquet --define_regions ${ILAMB_ROOT}/DATA/regions/LandRegions.nc ${ILAMB_ROOT}/DATA/regions/Whittaker.nc --regions global --model_setup model_setup.txt --filter .clm2.h0a`

### 5) After ILAMB has finished, run CUPiD diagnostics & webpage

   * `conda activate cupid-infrastructure`
   * `cupid-diagnostics -lnd`
   * `cupid-webpage`

**Note:** you can always remove a computed notebook & re-run cupid-diagnostics to get an updated version, e.g. if you run this before the LDF output exists and you want to rerun ILAMB later

### 6) View webpage:
  Can use another method, but it’s simple to scp the computed notebook directory locally and then open the index.html page


------
Code below was posted previously, but may not be relevent since ILAMB is now managed as an external to CUPiD?


In [ ]:
import os

from IPython.core.display import HTML, Image
from IPython.display import display

In [ ]:
ilamb_root = "."
base_case_name = None
start_date = ""
end_date = ""
base_start_date = None
base_end_date = None
key_plots = None
print_table = False
# ilamb_root will be external_diag_packages/computed_notebooks/ILAMB/

In [ ]:
# Want some base case parameter defaults to equal control case values
if base_case_name is not None:
    if base_start_date is None:
        base_start_date = start_date

    if base_end_date is None:
        base_end_date = end_date

In [ ]:
# convert start-date and end-date to year range
case_year_range = [int(start_date.split("-")[0]), int(end_date.split("-")[0]) - 1]

base_case_year_range = [
    int(base_start_date.split("-")[0]),
    int(base_end_date.split("-")[0]) - 1,
]

In [ ]:
display(
    HTML(
        '<a href="../ILAMB/index.html" target="_blank" style="font-size: 30px">Full ILAMB output</a>'
    )
)

## Key Metrics from ILAMB

Some important things to look at from ILAMB:

In [ ]:
for path_to_key_plot in key_plots:
    full_path = os.path.join(ilamb_root, path_to_key_plot)
    if os.path.isfile(full_path):
        print(full_path)
        display(Image(full_path))

In [ ]:
score_chart = os.path.join(ilamb_root, "index.html")

if print_table:
    display(HTML(f'<iframe src="{score_chart}"></iframe>'))